# Extract and Format MSMS Spectra from Raw Data
## Parameters
The next code block sets parameters that are used throughout the remainder of the notebook.

In [ ]:
# msms extraction metadata in CSV file format
msms_data_file = None

# name of new msms refs file
# name must end in ".tab"
msms_refs_file = None

# prefix to append to spectral ids
# if None, no prefix is added
id_prefix = None

# directory that contains raw data
raw_data_dir = None

# fragmentation method, this is typically 'HCD'
frag_method = None

# instrument type, this is typically 'Orbitrap'
instrument_type = None

# number of decimal places to round spectral data, default is 4.0
decimal = None

# The rest of this block contains project independent parameters

# to use an older version of the metatlas source code, set this to a commit id,
# branch name, or tag. If None, then use the the "main" branch.
source_code_version_id = None

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

## Initialization

In [ ]:
# pylint: disable=wrong-import-position,import-error,missing-class-docstring
import logging  # noqa: E402
from pathlib import Path  # noqa: E402
from IPython.display import Markdown, display  # noqa: E402


class StopExecution(Exception):
    def _render_traceback_(self):
        pass


logger = logging.getLogger("metatlas.jupyter")
kernel_def = """{"argv":["shifter","--entrypoint","--image=ghcr.io/biorack/metatlas/metatlas_shifter:latest","/usr/local/bin/python","-m",
                 "ipykernel_launcher","-f","{connection_file}"],"display_name": "Metatlas Targeted","language": "python",
                 "metadata": { "debugger": true }}"""
kernel_file_name = Path.home() / ".local" / "share" / "jupyter" / "kernels" / "metatlas-targeted" / "kernel.json"
try:
    has_root_kernel = Path("/root/.local/share/jupyter/kernels/papermill/kernel.json").is_file()
except PermissionError:
    has_root_kernel = False
if not has_root_kernel and not kernel_file_name.is_file():
    kernel_file_name.parent.mkdir(parents=True, exist_ok=True)
    with kernel_file_name.open(mode="w", encoding="utf-8") as f:
        f.writelines(kernel_def)
    logger.critical('CRITICAL: Notebook kernel has been installed. Set kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution
try:
    from metatlas.tools import notebook  # noqa: E402
except ImportError as err:
    logger.critical('CRITICAL: Set notebook kernel to "Metatlas Targeted" and re-run notebook.')
    raise StopExecution from err
notebook.setup(log_level, source_code_version_id)
from metatlas.tools.extract_msms import generate_msms_refs  # noqa: E402

## MSMS refs file generation

In [ ]:
generate_msms_refs(msms_data_file, 
                   msms_refs_file, 
                   id_prefix, 
                   raw_data_dir, 
                   frag_method, 
                   instrument_type, 
                   decimal)